In [ ]:
import os
import cv2
import pandas as pd
import numpy as np
from rich import print
import matplotlib.pyplot as plt
from IPython.display import display


from yolact_pkg.data.config import Config
from yolact_pkg.yolact import Yolact
from yolact_pkg.eval import infer, annotate_img, evaluate
# from yolact_pkg.train import train
# import torch

In [ ]:
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator


log_dir = "data_full/yolact/2022-05-02_kalo_qundis/training_2022-05-02-16꞉44/logs"

extract_data_from = ["train_epoch_total", "val_epoch_total"]

def tabulate_events(dpath):
    final_out = {}
    
    for folder_path in list(filter(os.path.isdir, [os.path.join(dpath,x) for x in os.listdir(dpath)])):
        folder = os.path.basename(folder_path)

        if extract_data_from is None or folder in extract_data_from:
    
            for dname in os.listdir(folder_path):
                ea = EventAccumulator(os.path.join(folder_path, dname)).Reload()
                tags = ea.Tags()['scalars']

                out = {}

                for tag in tags:
                    tag_values=[]
                    wall_time=[]
                    steps=[]

                    for event in ea.Scalars(tag):
                        tag_values.append(event.value)
                        wall_time.append(event.wall_time)
                        steps.append(event.step)
                        
                    out[tag] = pd.DataFrame(tag_values, columns=[tag], index=steps)

                if len(tags) == 1:
                    final_out[tags[0]] = out[tags[0]]
                else:
                    df = pd.concat(out.values())
                    final_out[dname] = df
    
    
    df_all = pd.concat(final_out.values(), axis=1)
    
    print(df_all.head())
    print(df_all.describe())
    print("min", df_all.min())
    print("idxmin", df_all.idxmin())
    
    idxmin = df_all.idxmin()
    print(idxmin["val/epoch"])
    val_min_row = df_all.loc[idxmin["val/epoch"]]
    print("validation min and corresponding train result", val_min_row)
    
    df_all.plot()
    
    # for key, df in final_out.items():
    #     # print(df)
    #     print("name", df.columns[0])
    #     display(df.head())
        
    #     # print(key)
    #     print(df.describe())
    #     print("min", df.min())
    #     print("idxmin", df.idxmin())
        
    #     df.plot()


    


tabulate_events(log_dir)